# Description

TODO

# Environment variables

In [ ]:
from IPython.display import display

import conf

N_JOBS = conf.GENERAL["N_JOBS"]
display(N_JOBS)

In [ ]:
%env MKL_NUM_THREADS=$N_JOBS
%env OPEN_BLAS_NUM_THREADS=$N_JOBS
%env NUMEXPR_NUM_THREADS=$N_JOBS
%env OMP_NUM_THREADS=$N_JOBS

# Modules loading

In [ ]:
%load_ext autoreload
%autoreload 2

In [4]:
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import umap
import matplotlib.pyplot as plt
import seaborn as sns

from utils import generate_result_set_name

# Settings

In [5]:
INITIAL_RANDOM_STATE = 12345

## Output directory

In [6]:
# # output dir for this notebook
# RESULTS_DIR = Path(
#     conf.RESULTS["CLUSTERING_RUNS_DIR"],
#     f"{INPUT_SUBSET}-{INPUT_STEM}",
# ).resolve()
# RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# display(RESULTS_DIR)

# Z-score standardized data

In [7]:
INPUT_SUBSET = "z_score_std"

In [8]:
INPUT_STEM = "projection-smultixcan-efo_partial-mashr-zscores"

In [9]:
input_filepath = Path(
    conf.RESULTS["DATA_TRANSFORMATIONS_DIR"],
    INPUT_SUBSET,
    f"{INPUT_SUBSET}-{INPUT_STEM}.pkl",
).resolve()
display(input_filepath)

assert input_filepath.exists(), "Input file does not exist"

input_filepath_stem = input_filepath.stem
display(input_filepath_stem)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/data_transformations/z_score_std/z_score_std-projection-smultixcan-efo_partial-mashr-zscores.pkl')

'z_score_std-projection-smultixcan-efo_partial-mashr-zscores'

In [10]:
data = pd.read_pickle(input_filepath)

In [11]:
data.shape

(3749, 987)

In [12]:
data.head()

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,...,LV978,LV979,LV980,LV981,LV982,LV983,LV984,LV985,LV986,LV987
100001_raw-Food_weight,-0.553743,1.851060,-0.095721,0.931603,-0.515196,1.371264,-0.413165,0.729668,0.546338,0.901153,...,1.148776,1.649018,-1.375952,2.203063,0.810199,-0.987048,0.277710,-0.607613,0.572376,-0.660846
100002_raw-Energy,-1.313905,-0.421015,-0.310524,-0.201978,0.233028,0.693838,0.084129,0.572439,0.217866,1.296973,...,0.165430,0.799276,-0.361989,-0.008253,2.063780,-0.930765,-0.147007,1.601938,-0.273553,-0.560822
100003_raw-Protein,-0.645622,-0.997845,0.249039,0.256499,0.422357,0.829189,-0.169004,0.511550,0.002389,-0.057200,...,1.016791,1.019488,-2.148844,-0.065130,0.041052,-0.836118,-0.320571,1.325519,-0.341494,-0.495786
100004_raw-Fat,-0.913924,-1.837183,0.012095,-1.301315,0.205794,1.129114,0.019985,0.376660,-0.810272,0.843611,...,0.544266,0.576714,0.281731,-0.218963,1.258957,-1.131924,0.378545,0.842047,-0.622194,-0.160058
100005_raw-Carbohydrate,-0.523004,0.131379,-0.097802,-0.030526,-0.021500,-0.219116,0.668229,0.365700,1.102347,0.845625,...,-0.344740,0.079274,0.253427,0.236474,1.033895,-0.229105,0.364793,1.776714,0.190531,-1.459641


## Data stats

In [13]:
data.min().min(), data.max().max()

(-9.173162070658442, 27.660674759843893)

In [14]:
assert not np.isinf(data).any().any()

In [15]:
assert not data.isna().any().any()

In [16]:
data_stats = data.describe()

In [17]:
data_stats.T

,count,mean,std,min,25%,50%,75%,max
LV1,3749.0,7.581144e-18,1.000133,-5.490319,-0.601540,-0.078123,0.493820,9.542784
LV2,3749.0,-7.581144e-18,1.000133,-3.290415,-0.648621,-0.047539,0.584184,8.482362
LV3,3749.0,7.581144e-18,1.000133,-2.366270,-0.590126,-0.140160,0.354790,8.700611
LV4,3749.0,-2.274343e-17,1.000133,-7.654516,-0.616646,-0.008664,0.589219,6.101977
LV5,3749.0,7.581144e-18,1.000133,-2.701507,-0.658095,-0.073984,0.501801,10.269057
...,...,...,...,...,...,...,...,...
LV983,3749.0,-3.411515e-17,1.000133,-4.204381,-0.645346,-0.079892,0.561935,13.183630
LV984,3749.0,-1.895286e-17,1.000133,-2.525851,-0.560073,-0.103051,0.370580,13.149904
LV985,3749.0,1.137172e-17,1.000133,-3.463111,-0.668643,-0.021685,0.613902,8.720886
LV986,3749.0,-3.790572e-18,1.000133,-5.625046,-0.621761,-0.009071,0.636834,5.718949


In [18]:
assert not np.isinf(data_stats).any().any()

In [19]:
assert not data_stats.isna().any().any()

## Check duplicated values

In [20]:
data_dups = data.duplicated()

In [21]:
data_dups.any()

True

In [22]:
data_dups.value_counts()

False    3725
True       24
dtype: int64

In [23]:
data_dups_labels = data.index[data_dups]
display(data_dups_labels)

Index(['C_DIGESTIVE_ORGANS', 'C_ENDOCRINE', 'C_EYE_BRAIN_NEURO',
       'C_FEMALE_GENITAL', 'C_LIP_ORAL_PHARYNX', 'C_MALE_GENITAL',
       'C_MESTOTHEL_SOFTTISSUE', 'C_RESPIRATORY_INTRATHORACIC', 'C_SKIN',
       'C_UNCERTAIN_SECONDARY', 'C_URINARY_TRACT',
       'H7_BCKRNDRETINOPAT-Background_retinopathy_and_retinal_vascular_changes',
       'I9_CHD_NOREV-Major_coronary_heart_disease_event_excluding_revascularizations',
       'I9_HEARTFAIL-Heart_failurestrict',
       'I9_HEARTFAIL_NS-Heart_failure_not_strict',
       'I9_HYPTENSPREG-Hypertension_complicating_pregnancy_childbirth_and_the_puerperium',
       'I9_MI_STRICT-Myocardial_infarction_strict',
       'ILD_DIFF_DG-ILD_differential_diagnosis', 'J10_ASTHMA-Asthma',
       'KRA_PSY_MOOD-Mood_disorders',
       'KRA_PSY_SCHIZODEL-Schizophrenia_or_delusion',
       'PNEUMONIA-Pneumonias_AsthmaCOPD_comorbidities',
       'PULM_ANXIETY-Anxiety_asthmarelated_comorbidities',
       'X_RESPIRATORY-Diseases_of_the_respiratory_system'],
 

# PCA

In [24]:
INPUT_SUBSET = "pca"

In [25]:
INPUT_STEM = "z_score_std-projection-smultixcan-efo_partial-mashr-zscores"

In [26]:
DR_OPTIONS = {
    "n_components": 50,
    "svd_solver": "full",
    "random_state": 0,
}

In [27]:
input_filepath = Path(
    conf.RESULTS["DATA_TRANSFORMATIONS_DIR"],
    INPUT_SUBSET,
    generate_result_set_name(
        DR_OPTIONS, prefix=f"{INPUT_SUBSET}-{INPUT_STEM}-", suffix=".pkl"
    ),
).resolve()
display(input_filepath)

assert input_filepath.exists(), "Input file does not exist"

input_filepath_stem = input_filepath.stem
display(input_filepath_stem)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/data_transformations/pca/pca-z_score_std-projection-smultixcan-efo_partial-mashr-zscores-n_components_50-random_state_0-svd_solver_full.pkl')

'pca-z_score_std-projection-smultixcan-efo_partial-mashr-zscores-n_components_50-random_state_0-svd_solver_full'

In [28]:
data = pd.read_pickle(input_filepath)

In [29]:
data.shape

(3749, 50)

In [30]:
data.head()

,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7,PCA8,PCA9,PCA10,...,PCA41,PCA42,PCA43,PCA44,PCA45,PCA46,PCA47,PCA48,PCA49,PCA50
100001_raw-Food_weight,-0.547515,-0.007250,0.245361,0.132424,1.201447,-0.113158,0.048859,1.031679,0.711208,0.126100,...,0.113685,-0.109753,0.178202,-0.487868,1.805920,-0.342611,-0.663712,-1.313987,-1.481707,0.396410
100002_raw-Energy,-1.570593,-0.618992,-1.223395,1.627321,-0.563860,0.224922,-1.500327,2.045633,-1.665998,-0.180432,...,0.697880,1.412676,2.103988,0.385625,2.386495,-2.939998,-0.978275,-1.893307,-0.259497,0.423264
100003_raw-Protein,-0.601212,-0.948824,-1.022209,2.438433,-2.062235,0.356202,-1.538038,2.314675,-1.108991,0.318034,...,0.171992,1.351979,0.054404,-0.725132,2.226520,-3.481674,-0.445294,-0.477452,0.237157,1.440867
100004_raw-Fat,-1.648214,-0.342042,-0.270189,1.683268,-1.222401,0.018349,-0.032824,1.671161,-0.908965,0.224100,...,-0.358812,0.696836,1.072624,0.627951,1.802000,0.174704,0.826319,-0.826992,0.596113,0.620050
100005_raw-Carbohydrate,-1.867586,-0.006412,-0.303220,0.770958,-0.116822,0.460946,-2.407732,1.534273,-1.930577,0.064621,...,1.332852,1.978032,2.158456,0.831123,1.093202,-3.539033,-0.831862,-0.842298,-0.723072,0.359438


## Data stats

In [31]:
data.min().min(), data.max().max()

(-49.816726369808826, 100.40836100619943)

In [32]:
assert not np.isinf(data).any().any()

In [33]:
assert not data.isna().any().any()

In [34]:
data_stats = data.describe()

In [35]:
data_stats.T

,count,mean,std,min,25%,50%,75%,max
PCA1,3749.0,3.638949e-16,8.485766,-5.743710,-3.131198,-2.434309,-0.815791,100.408361
PCA2,3749.0,-1.212983e-16,4.105379,-35.623760,-0.840904,-0.174942,0.552516,48.919206
PCA3,3749.0,7.581144e-18,3.294152,-38.575629,-0.664815,0.114516,0.863103,42.121356
PCA4,3749.0,1.516229e-17,3.094793,-49.816726,-0.610880,0.111067,0.803579,53.660260
PCA5,3749.0,0.000000e+00,3.013435,-24.146123,-0.924471,-0.189415,0.621831,28.486623
PCA6,3749.0,3.790572e-17,2.813783,-25.778786,-0.672630,0.081060,0.728776,90.377913
PCA7,3749.0,7.581144e-17,2.625236,-18.092530,-0.711175,0.057197,0.815881,37.031625
PCA8,3749.0,2.274343e-17,2.388907,-28.191215,-0.756626,0.012539,0.772162,41.857488
PCA9,3749.0,-7.581144e-18,2.381241,-35.214992,-0.732368,0.039457,0.784176,35.850059
PCA10,3749.0,-7.581144e-18,2.297545,-25.229730,-0.690982,0.152595,0.919323,25.529039


In [36]:
assert not np.isinf(data_stats).any().any()

In [37]:
assert not data_stats.isna().any().any()

## Check duplicated values

In [38]:
data_dups = data.duplicated()

In [39]:
data_dups.any()

True

In [40]:
data_dups.value_counts()

False    3725
True       24
dtype: int64

In [41]:
data.index[data_dups]

Index(['C_DIGESTIVE_ORGANS', 'C_ENDOCRINE', 'C_EYE_BRAIN_NEURO',
       'C_FEMALE_GENITAL', 'C_LIP_ORAL_PHARYNX', 'C_MALE_GENITAL',
       'C_MESTOTHEL_SOFTTISSUE', 'C_RESPIRATORY_INTRATHORACIC', 'C_SKIN',
       'C_UNCERTAIN_SECONDARY', 'C_URINARY_TRACT',
       'H7_BCKRNDRETINOPAT-Background_retinopathy_and_retinal_vascular_changes',
       'I9_CHD_NOREV-Major_coronary_heart_disease_event_excluding_revascularizations',
       'I9_HEARTFAIL-Heart_failurestrict',
       'I9_HEARTFAIL_NS-Heart_failure_not_strict',
       'I9_HYPTENSPREG-Hypertension_complicating_pregnancy_childbirth_and_the_puerperium',
       'I9_MI_STRICT-Myocardial_infarction_strict',
       'ILD_DIFF_DG-ILD_differential_diagnosis', 'J10_ASTHMA-Asthma',
       'KRA_PSY_MOOD-Mood_disorders',
       'KRA_PSY_SCHIZODEL-Schizophrenia_or_delusion',
       'PNEUMONIA-Pneumonias_AsthmaCOPD_comorbidities',
       'PULM_ANXIETY-Anxiety_asthmarelated_comorbidities',
       'X_RESPIRATORY-Diseases_of_the_respiratory_system'],
 

In [42]:
assert data.index[data_dups].equals(data_dups_labels)

# UMAP

In [43]:
INPUT_SUBSET = "umap"

In [44]:
INPUT_STEM = "z_score_std-projection-smultixcan-efo_partial-mashr-zscores"

In [45]:
DR_OPTIONS = {
    "n_components": 50,
    "metric": "euclidean",
    "n_neighbors": 15,
    "random_state": 0,
}

In [46]:
input_filepath = Path(
    conf.RESULTS["DATA_TRANSFORMATIONS_DIR"],
    INPUT_SUBSET,
    generate_result_set_name(
        DR_OPTIONS, prefix=f"{INPUT_SUBSET}-{INPUT_STEM}-", suffix=".pkl"
    ),
).resolve()
display(input_filepath)

assert input_filepath.exists(), "Input file does not exist"

input_filepath_stem = input_filepath.stem
display(input_filepath_stem)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/data_transformations/umap/umap-z_score_std-projection-smultixcan-efo_partial-mashr-zscores-metric_euclidean-n_components_50-n_neighbors_15-random_state_0.pkl')

'umap-z_score_std-projection-smultixcan-efo_partial-mashr-zscores-metric_euclidean-n_components_50-n_neighbors_15-random_state_0'

In [47]:
data = pd.read_pickle(input_filepath)

In [48]:
data.shape

(3749, 50)

In [49]:
data.head()

,UMAP1,UMAP2,UMAP3,UMAP4,UMAP5,UMAP6,UMAP7,UMAP8,UMAP9,UMAP10,...,UMAP41,UMAP42,UMAP43,UMAP44,UMAP45,UMAP46,UMAP47,UMAP48,UMAP49,UMAP50
100001_raw-Food_weight,12.424652,11.344468,10.228841,6.488212,12.847013,9.873152,2.202667,12.346815,8.819038,-1.838213,...,8.115669,6.076703,5.555357,4.784007,0.119412,10.161270,5.362766,1.907308,7.529003,3.673867
100002_raw-Energy,12.452178,11.338215,10.249873,6.471481,12.818970,9.868464,2.176878,12.315885,8.846581,-1.811907,...,8.122346,6.101641,5.545918,4.789267,0.101596,10.159430,5.400688,1.897933,7.510072,3.696232
100003_raw-Protein,12.412670,11.347814,10.219773,6.478085,12.838631,9.887061,2.214510,12.349669,8.812779,-1.834853,...,8.117552,6.070039,5.555128,4.786855,0.120158,10.150728,5.358839,1.919255,7.512418,3.664122
100004_raw-Fat,12.405792,11.343330,10.231809,6.499481,12.840716,9.851441,2.206928,12.337149,8.813656,-1.842742,...,8.118783,6.108067,5.576639,4.784761,0.127418,10.159600,5.397584,1.904287,7.542059,3.673757
100005_raw-Carbohydrate,12.415933,11.343335,10.238755,6.500062,12.844054,9.850930,2.201886,12.339456,8.812963,-1.844452,...,8.120839,6.105386,5.572872,4.789770,0.123977,10.164742,5.391287,1.900832,7.542199,3.676116


## Data stats

In [50]:
data.min().min(), data.max().max()

(-2.6016877, 14.177553)

In [51]:
assert not np.isinf(data).any().any()

In [52]:
assert not data.isna().any().any()

In [53]:
data_stats = data.describe()

In [54]:
data_stats.T

,count,mean,std,min,25%,50%,75%,max
UMAP1,3749.0,12.676297,0.606068,7.550988,12.570765,12.720392,12.877816,14.177553
UMAP2,3749.0,11.200081,0.551380,-0.219930,11.138938,11.238808,11.324599,11.698778
UMAP3,3749.0,10.654470,0.395270,4.529610,10.577783,10.692269,10.784398,11.115268
UMAP4,3749.0,6.643105,0.241658,5.386307,6.539877,6.678953,6.791796,7.205040
UMAP5,3749.0,12.951634,0.554749,8.829544,12.890648,13.049314,13.177445,13.532241
UMAP6,3749.0,9.286145,0.272683,7.674111,9.145744,9.279882,9.434588,10.049687
UMAP7,3749.0,1.961451,0.693667,1.347880,1.714486,1.842202,1.997281,7.150796
UMAP8,3749.0,11.968410,0.401654,9.193513,11.896320,12.035851,12.149180,12.699786
UMAP9,3749.0,9.023127,0.230475,8.031674,8.883487,8.999488,9.143533,9.807287
UMAP10,3749.0,-1.977741,0.360764,-2.601688,-2.159222,-2.057516,-1.910277,0.779222


In [55]:
assert not np.isinf(data_stats).any().any()

In [56]:
assert not data_stats.isna().any().any()

## Check duplicated values

In [57]:
data_dups = data.duplicated()

In [58]:
data_dups.any()

False

There are no duplicates with UMAP data, but the duplicates in `z_score_std` and `pca` are very close by in the UMAP representation.

In [59]:
data_dups_labels

Index(['C_DIGESTIVE_ORGANS', 'C_ENDOCRINE', 'C_EYE_BRAIN_NEURO',
       'C_FEMALE_GENITAL', 'C_LIP_ORAL_PHARYNX', 'C_MALE_GENITAL',
       'C_MESTOTHEL_SOFTTISSUE', 'C_RESPIRATORY_INTRATHORACIC', 'C_SKIN',
       'C_UNCERTAIN_SECONDARY', 'C_URINARY_TRACT',
       'H7_BCKRNDRETINOPAT-Background_retinopathy_and_retinal_vascular_changes',
       'I9_CHD_NOREV-Major_coronary_heart_disease_event_excluding_revascularizations',
       'I9_HEARTFAIL-Heart_failurestrict',
       'I9_HEARTFAIL_NS-Heart_failure_not_strict',
       'I9_HYPTENSPREG-Hypertension_complicating_pregnancy_childbirth_and_the_puerperium',
       'I9_MI_STRICT-Myocardial_infarction_strict',
       'ILD_DIFF_DG-ILD_differential_diagnosis', 'J10_ASTHMA-Asthma',
       'KRA_PSY_MOOD-Mood_disorders',
       'KRA_PSY_SCHIZODEL-Schizophrenia_or_delusion',
       'PNEUMONIA-Pneumonias_AsthmaCOPD_comorbidities',
       'PULM_ANXIETY-Anxiety_asthmarelated_comorbidities',
       'X_RESPIRATORY-Diseases_of_the_respiratory_system'],
 

In [60]:
data.loc[data_dups_labels]

,UMAP1,UMAP2,UMAP3,UMAP4,UMAP5,UMAP6,UMAP7,UMAP8,UMAP9,UMAP10,...,UMAP41,UMAP42,UMAP43,UMAP44,UMAP45,UMAP46,UMAP47,UMAP48,UMAP49,UMAP50
C_DIGESTIVE_ORGANS,12.522814,11.183684,10.430225,6.193108,12.772539,9.607761,2.360389,11.662754,9.596647,-1.880416,...,7.937743,6.121977,5.417243,4.780787,0.184049,10.319150,6.297142,1.325158,8.231253,3.512081
C_ENDOCRINE,12.627228,11.368789,10.635695,6.647035,12.885241,9.775416,1.937121,11.655753,9.168203,-1.766213,...,8.367372,6.583227,5.820481,4.925801,0.135162,10.430661,6.256723,1.311251,7.889904,3.518984
C_EYE_BRAIN_NEURO,12.746499,11.029016,10.591954,6.315557,12.738404,9.577576,2.132319,11.729331,9.285450,-1.644888,...,8.096117,6.199958,5.466055,4.859059,0.146200,10.319930,6.187963,1.464278,8.187665,3.584836
C_FEMALE_GENITAL,13.340240,11.407258,10.617926,6.982797,13.203407,9.390215,1.906018,12.275818,8.931524,-2.189009,...,8.011701,6.442483,5.324911,4.831779,-0.219365,10.329657,5.371190,1.471792,7.847337,3.677077
C_LIP_ORAL_PHARYNX,12.448047,11.194536,10.538993,6.755401,12.704965,9.003525,1.888897,11.959244,8.836426,-1.848705,...,7.955292,6.649662,5.766209,4.625515,0.292805,10.296896,5.778944,1.476414,7.893444,3.627465
C_MALE_GENITAL,13.064251,11.584308,10.368882,6.448215,12.792944,9.489837,1.406125,11.843698,9.147816,-2.044077,...,7.495809,6.387220,5.397659,4.507678,0.111709,10.986887,5.926845,1.579384,8.280014,3.635012
C_MESTOTHEL_SOFTTISSUE,12.766862,11.009689,10.798556,6.674001,12.841726,8.779061,1.548220,12.104630,9.012048,-2.018975,...,8.013562,6.620759,5.609308,4.745951,0.073742,10.415905,6.152078,1.532925,7.522064,3.488775
C_RESPIRATORY_INTRATHORACIC,12.675177,10.705385,10.502315,6.696950,12.623236,9.347958,3.029538,11.798676,8.617640,-1.627418,...,8.497452,6.971462,5.848913,4.721388,0.494897,10.233481,6.425020,1.279378,7.789337,3.567709
C_SKIN,13.313996,10.982435,10.572059,6.341154,12.879776,9.104074,2.135983,11.793115,8.932279,-1.487137,...,7.919952,6.347039,5.513787,4.839983,0.000993,10.324927,5.722449,1.514899,8.109125,3.439675
C_UNCERTAIN_SECONDARY,12.838195,11.507765,10.573780,6.683295,13.386426,9.119312,1.836551,12.226221,9.046413,-2.335533,...,7.794237,6.455453,5.512688,4.867457,0.007881,10.442236,5.931593,1.348678,7.873234,3.283968
